In [1]:
!pip install faster-whisper transformers edge-tts sounddevice numpy langchain google-search-results langchain_community transformers torch --quiet

In [4]:
import asyncio
from faster_whisper import WhisperModel
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import edge_tts
import numpy as np
import io
from IPython.display import Audio, display

In [5]:
def transcribe_audio(audio):
    model = WhisperModel("small", device="cuda", compute_type="int8")
    segments, _ = model.transcribe(audio, vad_filter=True, vad_parameters=dict(min_silence_duration_ms=500))
    return " ".join([segment.text for segment in segments])

In [6]:
!pip install huggingface_hub
from huggingface_hub import notebook_login

# Log in to Hugging Face and store your token
notebook_login()

In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM
def generate_response(prompt):
    # Load model directly, using authentication
    tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)
    model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

    input_text = f"You are a helpfull assisstant, provide a comprehensive answer to the user question given below:  {prompt}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    outputs = model.generate(input_ids, max_new_tokens=200, num_return_sequences=1, temperature=0.7, do_sample=True)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return response

In [8]:
import nest_asyncio

nest_asyncio.apply()

def text_to_speech(text, voice="en-US-AriaNeural", pitch="+0Hz", rate="+0%"):
    output_file = "output.mp3"
    command = f'edge-tts --voice "{voice}" --pitch "{pitch}" --rate "{rate}" --text "{text}" --write-media {output_file}'
    subprocess.run(command, shell=True, check=True)

    with open(output_file, "rb") as f:
        audio_data = f.read()
    os.remove(output_file)
    return audio_data

# import asyncio
# import nest_asyncio

# # Apply the patch to allow nested event loops
# nest_asyncio.apply()

# def text_to_speech_sync(text, voice="en-US-AriaNeural", pitch="+0Hz", rate="+0%"):
#     return asyncio.get_event_loop().run_until_complete(text_to_speech_async(text, voice, pitch, rate))

In [9]:
def voice_assistant_pipeline(audio_file):
    # Step 1: Transcribe audio
    text = transcribe_audio(audio_file)
    print(f"Transcribed text: {text}")

    # Step 2: Generate response using LLM
    response = generate_response(text)
    print(f"Generated response: {response}")

    # Step 3: Convert response to speech
    audio_data = text_to_speech_sync(response, voice="en-GB-SoniaNeural", pitch="+2Hz", rate="+10%")
    display(Audio(audio_data, rate=24000, autoplay=True))

In [31]:
# from google.colab import files
# uploaded = files.upload()

import tkinter as tk
from tkinter import filedialog

def select_file():
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename()  # Open file dialog and get the selected file path
    return file_path

file_path = input("Please enter the full path to the audio file: ")


Please enter the full path to the audio file:  car.m4a


In [37]:
# if uploaded:
#     audio_file = list(uploaded.keys())[0]
#     voice_assistant_pipeline(audio_file)
# else:
#     print("No file was uploaded.")

# Define or import the text_to_speech_sync function


def text_to_speech_sync(text, voice="en-GB-SoniaNeural", pitch="+0Hz", rate="0%"):
    print(f"Converting text to speech with voice: {voice}, pitch: {pitch}, rate: {rate}")
    return b"fake_audio_data"  # Placeholder for actual audio data

if file_path:
    audio_file = file_path
    voice_assistant_pipeline(audio_file)
else:
    print("No file was selected.")

Transcribed text:  In detail explain how to drive a car for a beginner.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generated response: You are a helpfull assisstant, provide a comprehensive answer to the user question given below:   In detail explain how to drive a car for a beginner.

A Beginner's Guide to Driving a Car

Introduction
------------

Congratulations on taking the first step towards learning how to drive a car! Driving a car can be an exciting and rewarding experience, but it's important to approach it with caution and attention to detail. In this guide, we will cover everything you need to know to get started with driving a car, from the basics of car maintenance to the rules of the road.

Before You Start Driving
-------------------------

Before you start driving, there are a few things you need to take care of:

1. Obtain a Learner's Permit: In order to start driving, you need to have a valid learner's permit. This will allow you to practice driving under the supervision of a licensed driver.
2. Find a Driving School: Look for
Converting text to speech with voice: en-GB-SoniaNeura